In [12]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
import nltk
from collections import Counter
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
lemmetizer  = WordNetLemmatizer()
with  open("train_ita_eng_spa_deu_fra.txt",'r',encoding='utf-8') as fp:
    language = []
    language_text  = []
    for line in fp:
        splitted_value  = line.split("\t")
        language.append(splitted_value[0].lower())
        myText = splitted_value[1].rstrip().lower()
        language_text.append(myText)
        


In [75]:
language_text[20]

'while running campaign elect his sister sherry martschink lieutenant governor shealy with the help robert kohn recruited unemployed black fisherman benjamin hunt run for congress against republican arthur ravenel shealy sought increase the turnout white voter playing the racial fear the south carolina electorate promised hunt who had prior drug arrest run for congress and paid for his filing fee shealy paid for the fee with unreported campaign contribution his sister from laidlaw environmental service'

In [14]:
def preprocessing_text(corpus):
    corpus_ = []
    for sentence in corpus:
        tr = str.maketrans("","",string.punctuation)
        sentence =  sentence.translate(tr)
        myCorpus = ""
        for word in sentence.split(" "):
            if not word.isdigit():
                if len(word) > 2:
                    
                    myCorpus += lemmetizer.lemmatize(word)+" "
        corpus_.append(myCorpus.rstrip())
    
    
    return corpus_

def bigram_words(splitted_sentence):
    return nltk.bigrams(splitted_sentence)
    
    


In [15]:
language_text  = preprocessing_text(language_text)
language_text = pd.Series(language_text)
language   =  pd.Series(language)

In [16]:
data  = np.stack([language_text,language],axis=1)
df = pd.DataFrame(data=data,columns=["text","label"])

In [17]:
for key,value  in enumerate(set(df["label"])):
    
    for name in range(len(df["label"])):
        if df.loc[name]["label"] ==value:
            df.loc[name]["label"] = key
              




In [18]:
def bigram_division(text_data):
    return list(nltk.bigrams(text_data))


def bigram_to_index(bigram_data):
    unique_bigram = set(bigram_data)
    b_to_index  = {v:k for k,v in enumerate(unique_bigram)}
    index_to_b  = {k:v for k,v in enumerate(unique_bigram)}
    
    return len(unique_bigram),b_to_index,index_to_b

def generateX(myBigramData):
    vector = np.zeros(unique_vocab_length)
    #generate the frequency of the word
    bigram_frequency = Counter(myBigramData)
    
    n_total = sum(bigram_frequency.values())
    
    for key,val in bigram_frequency.items():
        vector[b_to_index[key]] = val/n_total
        
    return vector

def tokenize(text_data):
    tokenized_text   = keras.preprocessing.text.text_to_word_sequence(text_data)
    return tokenized_text

In [19]:
#each row represent the vector and the vector is the unique value of the data 
text_data  = df["text"].values
text_data  = " ".join(text_data)

#how to find the unique value we need to divide the data into ngrams we can use the one hot encoding approach
tokenized_text   = keras.preprocessing.text.text_to_word_sequence(text_data)

bigram_data  = bigram_division(tokenized_text)
unique_vocab_length , b_to_index,index_to_b = bigram_to_index(bigram_data)

In [20]:
X  = []
for sentence in df["text"]:
    sentence  =  tokenize(sentence)
    big_ram = bigram_division(sentence)
    X.append(generateX(big_ram))
X  =  np.stack(X)
X.shape

(2500, 105776)

In [32]:
Y = df["label"]

In [63]:
Y = np.array(Y)
Y = Y.reshape(-1,1)


In [64]:
#lets create a model 

model = tf.keras.Sequential()

model.add(tf.layers.Dense(32,input_shape=(X.shape[1],) ,activation='relu'))
model.add(tf.layers.Dense(5,activation='softmax'))


model.compile(optimizer='adam',loss='sparse_categorical_crossentropy')
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 32)                3384864   
_________________________________________________________________
dense_27 (Dense)             (None, 5)                 165       
Total params: 3,385,029
Trainable params: 3,385,029
Non-trainable params: 0
_________________________________________________________________


In [65]:
model.fit(X,Y,epochs=10,batch_size=32)

Epoch 1/10
2500/2500 [==============================] - 14s 6ms/step - loss: 1.6066
Epoch 2/10
2500/2500 [==============================] - 1s 554us/step - loss: 1.5643
Epoch 3/10
2500/2500 [==============================] - 1s 562us/step - loss: 1.4576
Epoch 4/10
2500/2500 [==============================] - 1s 550us/step - loss: 1.2723
Epoch 5/10
2500/2500 [==============================] - 1s 572us/step - loss: 1.0341
Epoch 6/10
2500/2500 [==============================] - 1s 564us/step - loss: 0.7828
Epoch 7/10
2500/2500 [==============================] - 1s 578us/step - loss: 0.5605
Epoch 8/10
2500/2500 [==============================] - 1s 561us/step - loss: 0.3904
Epoch 9/10
2500/2500 [==============================] - 1s 591us/step - loss: 0.2727
Epoch 10/10
2500/2500 [==============================] - 1s 568us/step - loss: 0.1949


In [83]:
'''
1er avril le service asama sont actuellement effectués par de shinkansen série il 
étaient également effectués par 
de shinkansen série
'''

sentence =  ["1er avril le service asama sont actuellement effectués par de shinkansen série il étaient également effectués par de shinkansen série","while running campaign elect his sister sherry martschink lieutenant"]
X = []
for sent in sentence:
    sentence  =  tokenize(sent)
    big_ram = bigram_division(sentence)
    X.append(generateX(big_ram))

X = np.stack(X)

predicted_value  = model.predict(X)



In [89]:
output = {k:v for k,v in enumerate(set(language))}

for row in range(len(predicted_value)):
    print(output[np.argmax(predicted_value[row,:])])



fra
eng
